<a href="https://colab.research.google.com/github/patriciasandagorda/labo2025v/blob/main/src/ensembles/440_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

## 4.3 Random Forest

*Random Forest* es un algoritmo de ensembles de arboles de decision creado por Leo Brieman en 1995-2006
https://link.springer.com/content/pdf/10.1023/a:1010933404324.pdf

La página original es:
https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm

Dos buenos videos para seguir el paso a paso de Random Forest y aplicaciones:
* https://www.youtube.com/watch?v=J4Wdy0Wc_xQ
* https://www.youtube.com/watch?v=sQ870aTKqiM

Qué tipo de perturbaciones se realizan en Random Forest

*   Se perturba el dataset, con la técnica de bagging = Bootstrap Aggregating
*   Tambien se perturba el algoritmo, utiliza random en cada split

Cada arbolito de Random Forest se entrena sobre un dataset perturbado, que tiene :
* todas las columnas originales (esta es una GRAN diferencia con  Arboles Azarosos)
* la misma *cantidad* de registros del dataset original, PERO generados por la técnica de sampleo con reposición del dataset original.

A pesar de que Leo Brieman es también el inventor de CART (Classification and Regression Trees) Random Forest no corre el algoritmo CART de la libreria rpart, sino un CART perturbado, en donde cada split NO se hace sobre todos los campos del dataset, sino sobre un csubconjunto tomado al azar, esa cantidad es el hiperparámetro *mtry*

#### 4.3.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 4.4  Random Forest, una corrida

El tiempo de corrida de este punto es de alrededor de 8 minutos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 02 12:28:14 PM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660468,35.3,1454477,77.7,1438469,76.9
Vcells,1226772,9.4,8388608,64.0,1975128,15.1


**ranger** es una de las muchas librerías en lenguage R que implementa el algoritmo *Random Forest*, tiene la ventaja que corre el paralelo, utilizando todos los nucleos del procesador.

In [3]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

# ranger se usa para procesar
if( !require("ranger") ) install.packages("ranger")
require("ranger")

# randomForest  solo se usa para imputar nulos
if( !require("randomForest") ) install.packages("randomForest")
require("randomForest")

Loading required package: data.table

Loading required package: rpart

Loading required package: ranger

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘ranger’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘RcppEigen’


Loading required package: ranger

Loading required package: randomForest

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘randomForest’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: randomForest

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ranger’:

    importance




Aqui debe cargar SU semilla primigenia y

In [4]:
PARAM <- list()
PARAM$experimento <- 440
PARAM$semilla_primigenia <- 999961

PARAM$ranger$num.trees <- 300 # cantidad de arboles
PARAM$ranger$mtry <- 13 # cantidad de atributos que participan en cada split
PARAM$ranger$min.node.size <- 50 # tamaño minimo de las hojas
PARAM$ranger$max.depth <- 10 # 0 significa profundidad infinita


In [5]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KA", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [6]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [7]:
#  estas dos lineas estan relacionadas con el Data Drifting
# asigno un valor muy negativo

if( "Master_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Master_Finiciomora) , Master_Finiciomora := -999 ]

if( "Visa_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Visa_Finiciomora) , Visa_Finiciomora :=  -999 ]


In [8]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]

In [9]:
set.seed( PARAM$semilla_primigenia ) # Establezco la semilla aleatoria


# ranger necesita la clase de tipo factor
factorizado <- as.factor(dtrain$clase_ternaria)
dtrain[, clase_ternaria := factorizado]

In [10]:
# Ranger NO acepta valores nulos
# Leo Breiman, ¿por que le temias a los nulos?
# imputo los nulos, ya que ranger no acepta nulos
dtrain <- na.roughfix(dtrain)



In [11]:
setorder(dtrain, clase_ternaria) # primero quedan los BAJA+1, BAJA+2, CONTINUA

# genero el modelo de Random Forest llamando a ranger()
modelo <- ranger(
  formula= "clase_ternaria ~ .",
  data= dtrain,
  probability= TRUE, # para que devuelva las probabilidades
  num.trees= PARAM$ranger$num.trees,
  mtry= PARAM$ranger$mtry,
  min.node.size= PARAM$ranger$min.node.size,
  max.depth= PARAM$ranger$max.depth
)


Growing trees.. Progress: 20%. Estimated remaining time: 2 minutes, 4 seconds.
Growing trees.. Progress: 40%. Estimated remaining time: 1 minute, 31 seconds.
Growing trees.. Progress: 61%. Estimated remaining time: 1 minute, 0 seconds.
Growing trees.. Progress: 81%. Estimated remaining time: 28 seconds.


In [12]:
# Carpinteria necesaria sobre  dfuture
# como quiere la Estadistica Clasica, imputar nulos por separado
# ( aunque en este caso ya tengo los datos del futuro de antemano
#  pero bueno, sigamos el librito de estos fundamentalistas a rajatabla ...

dfuture[, clase_ternaria := NULL]
dfuture <- na.roughfix(dfuture)

In [13]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]

In [14]:
# aplico el modelo a los datos que no tienen clase
# aplico el modelo recien creado a los datos del futuro
prediccion <- predict(modelo, dfuture)

tb_prediccion[, prob := prediccion$predictions[, "BAJA+2"] ]

In [15]:
tb_prediccion[, Predicted := as.numeric(prob > (1/40))]

In [16]:
archivo_kaggle <- paste0("KA", PARAM$experimento,".csv")

# grabo el archivo
fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
 file= archivo_kaggle,
 sep= ","
)


In [17]:
# subida a Kaggle
comando <- "kaggle competitions submit"
competencia <- "-c labo-i-2025-virtual-analista-sr"
arch <- paste( "-f", archivo_kaggle)

mensaje <- paste0("-m 'num.trees=", PARAM$ranger$num.trees, "  mtry=", PARAM$ranger$mtry, "  min.node.size=", PARAM$ranger$min.node.size, " max.depth=", PARAM$ranger$max.depth, "'" )
linea <- paste( comando, competencia, arch, mensaje)
salida <- system(linea, intern= TRUE)
cat(salida)

Successfully submitted to LaboI 2025 virtual analista sr

In [18]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 02 12:34:13 PM 2025"



---



### 4.5  Random Forest  optimizacion de hiperparámetros

Random Forest es un algoritmo que quedó obsoleto luego de la aparición de  XGBoost y LightGBM, debido a lo lento de las librerías que lo implementan.
<br> El siguiente script se brinda simplemente a modo pedagógico, advirtiendo a los alumn@s que demanda más de 24 horas para correr, y los resultados son mediocres.

limpio el ambiente de R

In [19]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 02 12:34:13 PM 2025"

In [20]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1879071,100.4,3498482,186.9,3498482,186.9
Vcells,3384356,25.9,157950119,1205.1,164437876,1254.6


**ranger** es una de las muchas librerías en lenguage R que implementa el algoritmo *Random Forest*, tiene la ventaja que corre el paralelo, utilizando todos los nucleos del procesador.

In [21]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

# ranger se usa para procesar
if( !require("ranger") ) install.packages("ranger")
require("ranger")

# randomForest  solo se usa para imputar nulos
if( !require("randomForest") ) install.packages("randomForest")
require("randomForest")


if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


Loading required package: parallel

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes

Loading required package: rlist

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rlist’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘XML’


Loading required package: rlist

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

War

Aqui debe cargar SU semilla primigenia y

In [22]:
PARAM <- list()
PARAM$experimento <- 450
PARAM$semilla_primigenia <- 999961

PARAM$hyperparametertuning$iteraciones <- 100
PARAM$hyperparametertuning$xval_folds <- 5
PARAM$hyperparametertuning$POS_ganancia <- 117000
PARAM$hyperparametertuning$NEG_ganancia <- -3000

# Estructura que define los hiperparámetros y sus rangos
#  la letra L al final significa ENTERO
# max.depth 0 significa profundidad infinita
PARAM$hyperparametertuning$hs <- makeParamSet(
  makeIntegerParam("num.trees", lower= 20L, upper= 500L),
  makeIntegerParam("max.depth", lower= 1L, upper= 30L),
  makeIntegerParam("min.node.size", lower= 1L, upper= 1000L),
  makeIntegerParam("mtry", lower= 2L, upper= 50L)
)

In [23]:
# graba a un archivo los componentes de lista
# para el primer registro, escribe antes los titulos

loguear <- function(
    reg, arch= NA, folder= "./work/",
    ext= ".txt", verbose= TRUE) {

  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  if (!file.exists(archivo)) # Escribo los titulos
    {
      linea <- paste0(
        "fecha\t",
        paste(list.names(reg), collapse= "\t"), "\n"
      )

      cat(linea, file= archivo)
    }

  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t", # la fecha y hora
    gsub(", ", "\t", toString(reg)), "\n"
  )

  cat(linea, file= archivo, append= TRUE) # grabo al archivo

  if (verbose) cat(linea) # imprimo por pantalla
}


In [24]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30
# particionar( data=dataset, division=c(1,1,1,1,1),
#   agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar <- function(
    data, division, agrupa= "",
    campo= "fold", start= 1, seed= NA) {

  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by= agrupa
  ]
}


In [25]:
# es un paso del Cross Validation
# utiliza el fold  fold_test para testear y el resto para entrenar

ranger_Simple <- function(fold_test, pdata, param) {
  # genero el modelo

  set.seed(PARAM$semillas[2])

  modelo <- ranger(
    formula= "clase_binaria ~ .",
    data= pdata[fold != fold_test],
    probability= TRUE, # para que devuelva las probabilidades
    num.trees= param$num.trees,
    mtry= param$mtry,
    min.node.size= param$min.node.size,
    max.depth= param$max.depth
  )

  prediccion <- predict(modelo, pdata[fold == fold_test])

  ganancia_testing <- pdata[
    fold == fold_test,
    sum((prediccion$predictions[, "POS"] > 1 / 40) *
      ifelse(clase_binaria == "POS",
        PARAM$hyperparametertuning$POS_ganancia,
        PARAM$hyperparametertuning$NEG_ganancia
      ))
  ]

  return(ganancia_testing)
}


In [26]:
# realiza Cross Validation, promediando las ganancias de los folds de testing

ranger_CrossValidation <- function(
    data, param,
    pcampos_buenos, qfolds, pagrupa, semilla) {

  divi <- rep(1, qfolds)
  particionar(data, divi, seed= semilla, agrupa= pagrupa)

  ganancias <- mcmapply(ranger_Simple,
    seq(qfolds), # 1 2 3 4 5
    MoreArgs= list(data, param),
    SIMPLIFY= FALSE,
    mc.cores= 1
  ) # dejar esto en  1, porque ranger ya corre en paralelo

  data[, fold := NULL] # elimino el campo fold

  # devuelvo la ganancia promedio normalizada
  ganancia_promedio <- mean(unlist(ganancias))
  ganancia_promedio_normalizada <- ganancia_promedio * qfolds

  return(ganancia_promedio_normalizada)
}

In [27]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros se pasan como variables globales

EstimarGanancia_ranger <- function(x) {
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  xval_folds <- PARAM$hyperparametertuning$xval_folds

  ganancia <- ranger_CrossValidation(dataset,
    param= x,
    qfolds= xval_folds,
    pagrupa= "clase_binaria",
    semilla= PARAM$semillas[1]
  )

  # logueo
  xx <- x
  xx$xval_folds <- xval_folds
  xx$ganancia <- ganancia
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch= klog)

  # si es ganancia superadora la almaceno en mejor
  if( ganancia > GLOBAL_mejor ) {
    GLOBAL_mejor <<- ganancia
    loguear(xx, arch= klog_mejor)
  }


  return(ganancia)
}


aqui se inicia el programa

In [28]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [29]:
# genero numeros primos
primos <- generate_primes(min= 100000, max= 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, 2 )


In [30]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv", stringsAsFactors= TRUE)

In [31]:
dataset <- dataset[foto_mes %in% c(202107)]

In [32]:
#  estas dos lineas estan relacionadas con el Data Drifting
# asigno un valor muy negativo

if( "Master_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Master_Finiciomora) , Master_Finiciomora := -999 ]

if( "Visa_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Visa_Finiciomora) , Visa_Finiciomora :=  -999 ]


In [33]:
set.seed( PARAM$semilla_primigenia ) # Establezco la semilla aleatoria

In [34]:
# en estos archivos quedan los resultados
kbayesiana <- paste0("HT", PARAM$experimento, ".RDATA")
klog <- paste0("HT", PARAM$experimento, ".txt")
klog_mejor <- paste0("HT", PARAM$experimento, "_mejor.txt")

GLOBAL_iteracion <- 0 # inicializo la variable global
GLOBAL_mejor <- -Inf

# si ya existe el archivo log, traigo hasta donde llegue
if (file.exists(klog)) {
  tabla_log <- fread(klog)
  GLOBAL_iteracion <- nrow(tabla_log)
}


In [35]:
# paso a trabajar con clase binaria POS={BAJA+2}   NEG={BAJA+1, CONTINUA}
dataset[, clase_binaria :=
  as.factor(ifelse(clase_ternaria == "BAJA+2", "POS", "NEG"))]

dataset[, clase_ternaria := NULL] # elimino la clase_ternaria, ya no la necesito


In [36]:
# Ranger NO acepta valores nulos
# Leo Breiman, ¿por que le temias a los nulos?
# imputo los nulos, ya que ranger no acepta nulos

dataset <- na.roughfix(dataset)

In [37]:
# Aqui comienza la configuracion de la Bayesian Optimization

configureMlr(show.learner.output = FALSE)

funcion_optimizar <- EstimarGanancia_ranger

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar,
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hyperparametertuning$hs,
  has.simple.signature= FALSE
)

ctrl <- makeMBOControl(save.on.disk.at.time= 600, save.file.path= kbayesiana)

ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
)

ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


In [38]:
# inicio la optimizacion bayesiana

if (!file.exists(kbayesiana)) {
  run <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  run <- mboContinue(kbayesiana)
} # retomo en caso que ya exista

Growing trees.. Progress: 8%. Estimated remaining time: 5 minutes, 55 seconds.
Growing trees.. Progress: 17%. Estimated remaining time: 5 minutes, 15 seconds.
Growing trees.. Progress: 26%. Estimated remaining time: 4 minutes, 35 seconds.
Growing trees.. Progress: 35%. Estimated remaining time: 3 minutes, 59 seconds.
Growing trees.. Progress: 44%. Estimated remaining time: 3 minutes, 24 seconds.
Growing trees.. Progress: 53%. Estimated remaining time: 2 minutes, 51 seconds.
Growing trees.. Progress: 61%. Estimated remaining time: 2 minutes, 20 seconds.
Growing trees.. Progress: 70%. Estimated remaining time: 1 minute, 48 seconds.
Growing trees.. Progress: 77%. Estimated remaining time: 1 minute, 23 seconds.
Growing trees.. Progress: 86%. Estimated remaining time: 51 seconds.
Growing trees.. Progress: 95%. Estimated remaining time: 18 seconds.
Growing trees.. Progress: 8%. Estimated remaining time: 5 minutes, 52 seconds.
Growing trees.. Progress: 17%. Estimated remaining time: 5 minutes

[mbo] 91: num.trees=312; max.depth=22; min.node.size=613; mtry=27 : y = 5.5e+07 : 1960.6 secs : infill_ei

Saved the current state after iteration 92 in the file HT450.RDATA.

Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 996 points instead of 1000!”
Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 999 points instead of 1000!”


Growing trees.. Progress: 9%. Estimated remaining time: 5 minutes, 15 seconds.
Growing trees.. Progress: 18%. Estimated remaining time: 4 minutes, 40 seconds.
Growing trees.. Progress: 27%. Estimated remaining time: 4 minutes, 13 seconds.
Growing trees.. Progress: 36%. Estimated remaining time: 3 minutes, 42 seconds.
Growing trees.. Progress: 45%. Estimated remaining time: 3 minutes, 12 seconds.
Growing trees.. Progress: 54%. Estimated remaining time: 2 minutes, 41 seconds.
Growing trees.. Progress: 63%. Estimated remaining time: 2 minutes, 7 seconds.
Growing trees.. Progress: 72%. Estimated remaining time: 1 minute, 35 seconds.
Growing trees.. Progress: 81%. Estimated remaining time: 1 minute, 4 seconds.
Growing trees.. Progress: 90%. Estimated remaining time: 34 seconds.
Growing trees.. Progress: 99%. Estimated remaining time: 3 seconds.
Growing trees.. Progress: 9%. Estimated remaining time: 5 minutes, 32 seconds.
Growing trees.. Progress: 17%. Estimated remaining time: 4 minutes, 5

[mbo] 92: num.trees=469; max.depth=25; min.node.size=999; mtry=18 : y = 5.53e+07 : 1901.9 secs : infill_ei

Saved the current state after iteration 93 in the file HT450.RDATA.



Growing trees.. Progress: 16%. Estimated remaining time: 2 minutes, 40 seconds.
Growing trees.. Progress: 33%. Estimated remaining time: 2 minutes, 6 seconds.
Growing trees.. Progress: 50%. Estimated remaining time: 1 minute, 34 seconds.
Growing trees.. Progress: 66%. Estimated remaining time: 1 minute, 4 seconds.
Growing trees.. Progress: 83%. Estimated remaining time: 31 seconds.
Growing trees.. Progress: 100%. Estimated remaining time: 0 seconds.
Growing trees.. Progress: 16%. Estimated remaining time: 2 minutes, 40 seconds.
Growing trees.. Progress: 32%. Estimated remaining time: 2 minutes, 9 seconds.
Growing trees.. Progress: 49%. Estimated remaining time: 1 minute, 37 seconds.
Growing trees.. Progress: 65%. Estimated remaining time: 1 minute, 6 seconds.
Growing trees.. Progress: 81%. Estimated remaining time: 35 seconds.
Growing trees.. Progress: 98%. Estimated remaining time: 4 seconds.
Growing trees.. Progress: 16%. Estimated remaining time: 2 minutes, 43 seconds.
Growing trees

...

[mbo] 98: num.trees=249; max.depth=20; min.node.size=712; mtry=19 : y = 5.64e+07 : 1079.2 secs : infill_ei

Saved the current state after iteration 99 in the file HT450.RDATA.



Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 22 seconds.
Growing trees.. Progress: 37%. Estimated remaining time: 1 minute, 46 seconds.
Growing trees.. Progress: 55%. Estimated remaining time: 1 minute, 14 seconds.
Growing trees.. Progress: 74%. Estimated remaining time: 43 seconds.
Growing trees.. Progress: 92%. Estimated remaining time: 12 seconds.
Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 17 seconds.
Growing trees.. Progress: 37%. Estimated remaining time: 1 minute, 48 seconds.
Growing trees.. Progress: 55%. Estimated remaining time: 1 minute, 17 seconds.
Growing trees.. Progress: 73%. Estimated remaining time: 46 seconds.
Growing trees.. Progress: 92%. Estimated remaining time: 13 seconds.
Growing trees.. Progress: 18%. Estimated remaining time: 2 minutes, 20 seconds.
Growing trees.. Progress: 36%. Estimated remaining time: 1 minute, 49 seconds.
Growing trees.. Progress: 54%. Estimated remaining time: 1 minute, 20 seconds.
Growing tr

[mbo] 99: num.trees=370; max.depth=19; min.node.size=897; mtry=11 : y = 5.55e+07 : 950.8 secs : infill_ei

Saved the current state after iteration 100 in the file HT450.RDATA.



Growing trees.. Progress: 75%. Estimated remaining time: 10 seconds.
Growing trees.. Progress: 71%. Estimated remaining time: 12 seconds.
Growing trees.. Progress: 71%. Estimated remaining time: 12 seconds.
Growing trees.. Progress: 71%. Estimated remaining time: 12 seconds.
Growing trees.. Progress: 73%. Estimated remaining time: 11 seconds.
20251102 160346	52	21	720	19	5	54435000	116


[mbo] 100: num.trees=52; max.depth=21; min.node.size=720; mtry=19 : y = 5.44e+07 : 244.0 secs : infill_ei

Saved the final state in the file HT450.RDATA



In [39]:
# analizo la salida de la bayesiana

tb_bayesiana <- fread(klog)
setorder( tb_bayesiana, -ganancia)
tb_bayesiana

fecha,num.trees,max.depth,min.node.size,mtry,xval_folds,ganancia,iteracion
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
20251102 003409,447,18,397,10,5,56532000,81
20251102 154350,249,20,712,19,5,56403000,114
20251102 065233,500,13,460,16,5,56289000,98
20251102 140144,402,28,761,11,5,56256000,109
20251101 141908,377,22,551,10,5,56244000,61
20251101 060148,500,11,481,29,5,56241000,51
20251102 024001,479,19,356,10,5,56235000,86
20251101 153441,371,27,581,9,5,56226000,65
20251102 021714,469,19,411,9,5,56220000,85


In [40]:
# mejores parametros

print( tb_bayesiana[1] )

             fecha num.trees max.depth min.node.size  mtry xval_folds ganancia
            <char>     <int>     <int>         <int> <int>      <int>    <int>
1: 20251102 003409       447        18           397    10          5 56532000
   iteracion
       <int>
1:        81


In [41]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 02 04:03:47 PM 2025"



---

